In [1]:
from export.nb_01 import *

D:\git\dl\dev\export\nb_01.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
D:\narain\anaconda3\envs\torch\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
get_mnist??

Signature: get_mnist()
Docstring: <no docstring>
Source:   
def get_mnist():
    with gzip.open(mnist_path, 'rb') as f:
        ((x_train, y_train),(x_valid, y_valid), _)=pickle.load(f, encoding='latin-1')
    x_train, y_train, x_valid, y_valid= map(tensor, (x_train, y_train, x_valid, y_valid))
    return (x_train, y_train), (x_valid, y_valid)
File:      d:\git\dl\dev\export\nb_01.py
Type:      function


In [3]:
(x_train, y_train), (x_valid, y_valid)= get_mnist()

In [5]:
train_ds, valid_ds= Dataset(x_train, y_train), Dataset(x_valid, y_valid)
c=y_train.max().item()
loss_fn=F.cross_entropy

In [7]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl, self.valid_dl, self.c= train_dl, valid_dl, c
        
    @property
    def train_ds(self): return self.train_dl.dataset
    
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [8]:
#export
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [14]:
data=DataBunch(*get_dls(train_ds, valid_ds, 128),c)

In [15]:
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

In [17]:
learn = Learner(*get_model(data), loss_fn, data)

In [18]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb,yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += learn.loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(learn.data.valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [20]:
fit(1, learn)

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at C:\w\1\s\tmp_conda_3.7_104508\conda\conda-bld\pytorch_1572950778684\work\aten\src\THNN/generic/ClassNLLCriterion.c:97